<a href="https://colab.research.google.com/github/Naomi-NLP/English_Yoruba_Machine_Translation/blob/main/English_Yoruba_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 37.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.0
    Uninstalling transformers-5.0.0:
      Successfully uninstalled transformers-5.0.0


In [2]:
!pip install transformers sentencepiece sacremoses
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.4 MB/s eta 0:00:00


In [3]:
# Load your CSV
import pandas as pd
MT_data = "/content/drive/MyDrive/MT.csv"
df = pd.read_csv(MT_data)

# Check first rows
df.head()
df.columns

Index(['English', 'Yoruba'], dtype='object')

In [5]:
df.shape

(6633, 2)

In [7]:
# Randomly sample 300 rows from the dataframe for analysis
df_sample = df.sample(n=300, random_state=42).reset_index(drop=True)
df_sample.shape

(300, 2)

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from tqdm import tqdm
import os

In [11]:
torch.cuda.is_available()

True

In [13]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("translation", model="facebook/nllb-200-distilled-1.3B")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

KeyError: "Invalid translation task translation, use 'translation_XX_to_YY' format"

In [12]:
model_name = "facebook/nllb-200-distilled-600M"
src_lang = "eng_Latn"
tgt_lang = "yor_Latn"

In [16]:
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang=src_lang)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

Loading model...


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [17]:
def translate_batch(texts, batch_size=8):
    translations = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
        batch = [str(t) if pd.notna(t) else "" for t in texts[i:i+batch_size]]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        translated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer._convert_token_to_id_with_added_voc(tgt_lang),
            max_length=512
        )
        batch_translations = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        translations.extend(batch_translations)
    return translations


In [18]:
def process_file(input_path, output_path1, output_path2):
    print(f"Processing {input_path}...")
    df_sample = pd.read_csv(input_path, sep=",")

In [19]:
df_translated = df_sample.copy()

print("Translating english column...")

# Translate the english column
df_translated["Yoruba_translated_model_output"] = translate_batch(df_sample["English"].tolist(), batch_size=8)

# Define the output path
output_path = "/content/drive/MyDrive/translated_MT.csv"

# Save to CSV
df_translated.to_csv(output_path, index=False)

print(f"Saved translated file {output_path}")

Translating english column...


Translating: 100%|██████████| 38/38 [01:38<00:00,  2.59s/it]


Saved translated file /content/drive/MyDrive/translated_MT.csv


In [22]:
df1 = pd.read_csv("/content/drive/MyDrive/translated_MT.csv")
df1.head(20)

,English,Yoruba,Yoruba_translated_model_output
0,"Otherwise, they would burn down all the buildi...","Bí bẹ́ẹ̀kọ́, wọn á dáná sun gbogbo ilé inú àgọ...","bí kò bá rí bẹ́ẹ̀, wọ́n á dáná sun gbogbo ilé ..."
1,"In Turkey, 2.5m students sit university entry ...","Ní orílẹ̀-èdè Turkey, ẹgbẹẹgbẹ̀rún méjì àbọ̀ à...","Ni Tọki, awọn ọmọ ile-iwe 2.5m joko ni idanwo ..."
2,"51. And finally, I will thank all Nigerians ag...","51.Nípàárí, màá tún dúpẹ́ lọ̣́wọ́ gbogbo àwọ̣̣...","Níkẹyìn, mo tún fẹ́ dúpẹ́ lọ́wọ́ gbogbo ọmọ or..."
3,America is so beautiful.,Ìlú Amẹ́ríkà mà rẹwà.,Amẹ́ríkà lẹ́wà gan - an.
4,And now she didn't know whether to continue wi...,"Nísinsìnyí, kò mọ̀ bóyá kí ó tẹ̀síwájú pẹ̀lú ọ...","Ní báyìí, ó ti ń sọ pé òun ò mọ̀ bóyá òun á tẹ..."
5,You can change this in the settings.,O lè yí èyí padà nínú ààtò.,O le yi eyi pada ni awọn eto.
6,The United nations has said that incurable dis...,Àjọ àgbáyé ti ní ààrùn kògbóògùn HIV/AIDS ní í...,Orílẹ̀-èdè Àjọ Ìparapòpọ̀ ti sọ pé àrùn àìlera...
7,"See Labake flopping in an ""under-the-roof cros...",Ẹ wo bí Làbákẹ́ ṣe ń koṣé níbi ìfọ̀rọ̀wánilẹ́n...,"Wo Labake ti n ṣubu ni ""ẹri-ifowopamọ labẹ-ifo..."
8,"However, the use of an image from a different ...","Àmọ́ ṣá, àwòrán orílẹ̀-èdè mìíràn tí ó lò láti...","àmọ́, lílo àwòrán kan láti orílẹ̀ - èdè mìíràn..."
9,---Germany has stated its readiness to partner...,---Orílẹ̀ èdè Germany ti ní ohun ṣetán láti fọ...,Jẹmánì ti ṣalaye igbaradi rẹ lati ṣe alabapin ...


In [24]:
df1.shape

(300, 3)

Evaluation

In [25]:
!pip install sacrebleu evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.9 MB/s eta 0:00:00


In [26]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00


In [27]:
import evaluate

# Load metrics
bleu = evaluate.load("sacrebleu")
meteor = evaluate.load("meteor")
bertscore = evaluate.load("bertscore")

# References and predictions
predictions = df["Yoruba_translated_model_output"].tolist()
references = [[ref] for ref in df["Yoruba"].tolist()]



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [29]:
bleu_refs = [[r[0]] for r in references]
flat_refs = [r[0] for r in references]

# BLEU
bleu_score = bleu.compute(predictions=predictions, references=bleu_refs)
print("BLEU:", bleu_score)

# METEOR
meteor_score = meteor.compute(predictions=predictions, references=flat_refs)
print("METEOR:", meteor_score)

# BERTScore
bertscore_score = bertscore.compute(predictions=predictions, references=flat_refs, lang="en")
print("BERTScore:", bertscore_score)

BLEU: {'score': 8.260798988998564, 'counts': [2684, 877, 377, 194], 'totals': [8254, 7954, 7655, 7356], 'precisions': [32.517567240126, 11.025898918783001, 4.924885695623775, 2.6373028820010878], 'bp': 1.0, 'sys_len': 8254, 'ref_len': 7418}
METEOR: {'meteor': np.float64(0.2857196929368024)}
BERTScore: {'precision': [0.9353638291358948, 0.8177362680435181, 0.909000813961029, 0.910542905330658, 0.8866944909095764, 0.8006694316864014, 0.8861954808235168, 0.8435543775558472, 0.9038501977920532, 0.860317587852478, 0.8428210020065308, 0.8865384459495544, 0.8848045468330383, 0.8879620432853699, 0.878430962562561, 0.9129273891448975, 0.9323664903640747, 0.9179236888885498, 0.8335167169570923, 0.8926639556884766, 0.8985641002655029, 0.8521175384521484, 0.8448473215103149, 0.9147299528121948, 0.8859317302703857, 0.9139065742492676, 0.9017369747161865, 0.883816123008728, 0.9002390503883362, 0.877335250377655, 0.9153804779052734, 0.8741843104362488, 0.9054568409919739, 0.8990469574928284, 0.891364